## Step 2 - Database Engineering

Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.

* Create a Jupyter Notebook called `database_engineering.ipynb` and use this to complete all of your Database Engineering work.

* Use Pandas to read your cleaned measurements and stations CSV data.

* Use the `engine` and connection string to create a database called `hawaii.sqlite`.

* Use `declarative_base` and create ORM classes for each table.

  * You will need a class for `Measurement` and for `Station`.

  * Make sure to define your primary keys.

* Once you have your ORM classes defined, create the tables in the database using `create_all`.

In [ ]:
# Dependencies
import pandas as pd
from datetime import datetime
import sqlalchemy
from sqlalchemy import create_engine, MetaData #connect to database
from sqlalchemy.ext.declarative import declarative_base #classes into tables
from sqlalchemy import Column, Integer, String, Float, DateTime #allow us to declare column typs
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import pymysql

pymysql.install_as_MySQLdb() # starts creating connection from Python to SQL database

In [ ]:
# Create a reference the CSV file
measurementFile = 'resources/clean_hawaii_measurements.csv'
stationFile = 'resources/clean_hawaii_stations.csv'

# Load the cleaned csv file into a pandas dataframe
measurementDF = pd.read_csv(measurementFile)
stationDF = pd.read_csv(stationFile)

# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
measurementData = measurementDF.to_dict(orient='records')
stationData = stationDF.to_dict(orient='records')

In [ ]:
# Create our database engine
engine = create_engine('sqlite:///hawaii.sqlite')
conn = engine.connect()

# Create base
Base = declarative_base() #object that utilizes the default for declarative base

# Create ORM classes
class Measurement(Base):
    __tablename__='measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(DateTime)
    prcp = Column(Float(5,5))
    tobs = Column(Integer)
    
    def __repr__(self):
        return f'id={self.id},station={self.station},date={self.date},prcp={self.prcp},tobs={self.tobs}'

#format='%Y-%m-%d'
    
class Station(Base):
    __tablename__='station'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float(3,5))
    longitude = Column(Float(3,5))
    elevation = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id},station={self.station},name={self.name},latitude={self.latitude},longitude={self.longitude},elevation={self.elevation}"

In [ ]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()
    
# Use a Session to test the classes
Base.metadata.create_all(engine)
session = Session(bind=engine)

# Save the reference to the sqlite tables as a variables
measurementTable = sqlalchemy.Table('measurement', metadata, autoload=True)
stationTable = sqlalchemy.Table('station', metadata, autoload=True)

In [ ]:
# Use `table.delete()` to remove any pre-existing data.
# conn.execute(measurementTable.delete())
conn.execute(stationTable.delete())

In [ ]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(measurementTable.insert(), measurementData)
conn.execute(stationTable.insert(), stationData)

In [ ]:
measurementDF.head()